<a href="https://colab.research.google.com/github/ferkrum/DS-Notebooks/blob/main/Aula_28b_ferkrum_twitter_%2B_dict_lexico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DESAFIO:
* Utilizando a api do Twitter:
  * ~~escolha um usuário ou tema (pt-br)~~
  * ~~utilize o dicionário léxico da última aula~~
  * ~~aplicar um classificador de sentimentos, nos tweets obtidos~~

* Problemas que irão encontrar:
  * Problemas na escrita

* Técnicas (tanto nos tweets quanto no dicionário):
  * ~~colocar todas as palavras em minúsculo~~
  * remover stopwords
  * ~~remover todos os acentos~~
  * ~~transformar palavras como vc em você~~
  * plus: utilizar apenas o radical (lemma, stemming)

# Obtendo dicionário lexico (lexiconPT)

In [17]:
URL_RAW = 'https://raw.githubusercontent.com/sillasgonzaga/lexiconPT/master/data-raw/SentiLex-lem-PT02.txt'

#importando dicionario
import requests
response = requests.get(URL_RAW)
#usando o método .text para obter o texto de response
response = requests.get(URL_RAW).text
#quebrando nas novas linhas. transforma a string em um array
response = response.split('\n')

## Montando o dicionário lexico

In [18]:
dict = {}
for x in response:
  palavra = x.split(".")[0]
  peso = x.split(";")[2].split("=")[1]
  dict[palavra]=int(peso)

In [25]:
#modelo de request
dict.get('abraco',0)

0

# Funções para filtrar texto

In [78]:
import re #regex
import string #para remover pontuações (ref: https://www.geeksforgeeks.org/python-extract-words-from-given-string/)

In [81]:
#instalando unidecode para remover acentos conforme (https://www.geeksforgeeks.org/how-to-remove-string-accents-using-python-3/?ref=gcse)
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 31.8 MB/s 


In [88]:
import unidecode as uni #para remover acentos da string

In [92]:
#ref de: https://www.geeksforgeeks.org/python-extract-words-from-given-string/

def limpa_texto(s):
  s = uni.unidecode(s) #remove acentos
  result = re.sub('['+string.punctuation+']', '', s.lower()).split()  
  return result

In [94]:
#teste
limpa_texto('coração melão çãõ')

['coracao', 'melao', 'cao']

In [ ]:
#dicionario para converter abreviações de palavras em palavras completas


## Dicionário para substituir abreviações

In [143]:
dictAbrev = {'vc': 'voce', 
         'td': 'tudo', 
         'abl': 'abelha',
         'qt': 'quanto' }
def limpa_abrev(s):
  return dictAbrev.get(s,s)


In [132]:
limpa_abrev('vc')

'voce'

# Classificador Léxico (função)

In [150]:
#copiando classificador da aula passada:
def classificador(s):
  s = limpa_texto(s) #prepara o texto. já entrega o texto em uma lista
  soma = 0
  for palavra in s:
    palavra = limpa_abrev(palavra) #substitui as abreviacoes pelas palavras
    polaridade = dict.get(palavra,0)
    
    print(palavra,'=', polaridade)
    
    soma += polaridade #percorre a lista s.split() usando a variavel "palavra". 
                                        #acumula na variável soma os valores de cada palavra. 
                                        #caso não encontre, soma "0" à variável.
  print(soma)
  if soma > 0:
    return 'positivo'
  elif soma == 0:
    return 'neutro'
  else:
    return 'negativo'

In [145]:
# testando classificador
classificador('miga, vc gosta de qt bondade?')

miga = 0
voce = 0
gosta = 0
de = 0
quanto = 0
bondade = 1
1


'positivo'

# Codigo para conexão com Twitter:

In [ ]:
!pip install tweepy
import tweepy as tw 

#dados anderson
consumer_key = 'bZD6d3QYbsgqwkteNGoqfbQPp'
consumer_secret = 'r6nLWRrSH9HDizbB39n2DqLbLAcZWCqDsSQtpSTHO2azNIyjae'
access_token = '2560618594-xNBFHggcdFA3S2ntNY1hYqudT37W0AdZaBa2jZ9'
access_token_secret = 'ExSkQ9KcXxsLhGBHmmYvRDUX0IA9q6grdMb294WCAbscI'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAJz7YAEAAAAA%2FWRbWUJQnGLsm6zLX1UFnpld2cs%3DYroih3cDjMmcQnZDHuFm0STxsLGRov7vYhtlhq89KmeXU8TrPg'

#modelo de autenticação do Anderson:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Query para o twitter:

In [146]:
#escolhi o Randolfe Rodrigues porque hoje foi aprovada a CPI do MEC. (28/jun/2022)
tweets = api.user_timeline(screen_name='randolfeap', count=10)

In [101]:
for tweet in tweets:
  print(tweet.text.lower(),'\n')

cpi do mec protocolada! vamos ao trabalho! #cpidomec https://t.co/rb02wdchrv 

agora contamos com 31 assinaturas, após o registro do senador @jarbas156. a tendência é aumentar! continuamos na lu… https://t.co/efvnn1p2bw 

acabamos de protocolar o requerimento da #cpidomec. o protocolo ocorreu com 30 assinaturas, após o registro da assi… https://t.co/2ytmzcg51r 

atenção! logo mais, às 12h, vamos protocolar o requerimento para a criação da cpi do mec. em seguida, iremos conced… https://t.co/ezzvnjlix0 

é um absurdo, por exemplo, que qualquer instituição se preste ao papel de interferir no nosso processo democrático,… https://t.co/xmhkjyqout 

a fome, o desemprego, a carestia, a inflação, nada disso é um processo natural. é um processo político. quando o po… https://t.co/aixydhltjd 

atenção! acabo de acionar o stf para que o celular do ex-presidente da petrobras seja imediatamente apreendido e pe… https://t.co/hi4r4jkk1u 

precisamos passar a limpo toda corrupção desse governo!

todos 

# Teste tweets

In [151]:
tweets[0].text

'Depois de uma semana de notícias devastadoras sobre violência sexual, surge mais uma! São graves as acusações contr… https://t.co/AN6wZFRQ0w'

# Teste classificador

In [154]:
#teste classificador
classificador('violento')

violento = -1
-1


'negativo'

In [115]:
classificador(tweets[8].text)

-2


'negativo'

In [ ]:
#Aplicando dentro do Loop For: 
for tweet in tweets:
  print(tweet.text,'\n')